In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial
import pickle, os, time
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import entropy
import loaddata

#basedir, desc = 'saveddata3/Regression/v1', 'Regression'
#basedir, desc = 'saveddata4/Regression/v2', 'Regression'
#basedir, desc = 'saveddata/MNIST/v1', 'MNIST'
#basedir, desc = 'saveddata3/NoisyClassifier/v1', 'NoisyClassifier'
#basedir, desc = 'saveddata3/NoisyClassifierWine/v2', 'NoisyClassifierWine'
#basedir, desc = 'saveddata3/NoisyClassifierWine/v3d', 'NoisyClassifierWineV3'
#basedir, desc = 'saveddata3/NoisyClassifierWine/v13', 'NoisyClassifierWineV10'
# basedir, desc = 'saveddata4/NoisyClassifierWine/v1', 'NoisyClassifierWineV10'

#basedir, desc = 'saveddata-scipy/NoisyClassifier/v1', 'NoisyClassifier'
basedir, desc = 'saveddata-scipy/MNIST/v1', 'MNIST'



In [ ]:
vals = {}
vals['VIB']         = {}
vals['VIB']['reg']  = {'tst': {}, 'trn': {}}
vals['VIB']['sq']   = {'tst': {}, 'trn': {}}
vals['nlIB']        = {}
vals['nlIB']['reg'] = {'tst': {}, 'trn': {}}
vals['nlIB']['sq']  = {'tst': {}, 'trn': {}}

In [ ]:
n_average_epoch = 10  # Average results for these last n epochs, to smooth
runtype = None
data    = None

for fname in os.listdir(basedir):
    if not fname.startswith('results-'):
        continue
    print(basedir + '/' + fname)
    with open(basedir + '/' + fname + '/data', 'rb') as f:
        try:
            cfg, saved_data = pickle.load(f)
        except:
            print("Error loading pickle file")
            continue
        
    if runtype is None:
        runtype = cfg['runtype']
        data    = loaddata.load_data(runtype)

    elif runtype != cfg['runtype']:
        raise Exception('Mixed runtypes!')

    sqmode = 'sq' if cfg['squaredIB'] else 'reg'
    beta   = saved_data[0]['beta']
    mode   = saved_data[0]['mode']
    #if False and 'loss' in saved_data[0]['trn']:
    #    losses = np.array([r['trn']['loss'] for r in saved_data])
    #    lrow = saved_data[len(losses)-best_of_last+np.argmin(losses[-best_of_last:])]
    #else:
    #    lrow = saved_data[-1]
    #_, mode, *_ = fname.split("-",)
    for r in ['trn','tst']:
        val_dict = {}
        permutation  = np.random.permutation(len(data[r+'_Y']))
        Xbatch = data[r+'_X'][permutation][:2000]
        tf.reset_default_graph()
        with tf.Session() as sess:
            try:
                saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('%s/%s' % (basedir, fname))+'.meta')
                for lrow in saved_data[-n_average_epoch:]:
                    var   = lrow['noisevar']
                    l     = lrow[r]
                    epoch = lrow['epoch']

                    saver.restore(sess, '%s/%s/tf_model-%d' % (basedir, fname, epoch))
                    mx = sess.run('T_nonoise:0', feed_dict={'X:0':Xbatch})

                    n, d  = mx.shape
                    mcH   = entropy.get_mc_entropy(mx, var)
                    hCond = entropy.gaussian_entropy_np(d, var)
                    l['Ixt_mc'] = mcH - hCond
                    for k in ['Ixt','Ixt_lb','Iyt','Ixt_mc']:
                        if k not in val_dict:
                            val_dict[k]=0.
                        val_dict[k] += l[k]/n_average_epoch
            except:
                print("Error loading tensorflow model")
                continue

        print('%s %s beta: %4g Ixt: %0.2f-%0.2f (%0.2f) Iyt: %0.2f' % 
              (r, sqmode, beta, val_dict['Ixt_lb'], val_dict['Ixt'], val_dict['Ixt_mc'], val_dict['Iyt']))

        if beta not in vals[mode][sqmode][r] or \
           (val_dict['Ixt_mc'] <= vals[mode][sqmode][r][beta]['Ixt_mc'] and 
            val_dict['Iyt']    >= vals[mode][sqmode][r][beta]['Iyt']):
            vals[mode][sqmode][r][beta] = val_dict
            if False:
                plt.figure()
                plt.scatter(mx[:,0], mx[:,1], s=l['noisevar'], alpha=.3)
                plt.title('beta=%g mode=%s'%(beta, mode))
    print()

In [ ]:
import seaborn as sns
sns.set_style('white')
sns.set(font_scale=1.25)
colordict={('nlIB','reg'):'blue', ('nlIB','sq'):'blue', ('VIB','reg'):'green', ('VIB','sq'):'green'}

plt.figure(figsize=(12,5))
for rndx, r in enumerate(['trn','tst']):
    for sqmode in ['reg','sq']:
        plt.subplot(1,2,rndx+1)
        for mode in ['nlIB', 'VIB']:
            d     = vals[mode][sqmode][r]
            bvals = sorted(d.keys())
            xt    = [d[b]['Ixt'] for b in bvals]
            xtlb  = [d[b]['Ixt_lb'] for b in bvals]
            xtmc  = [d[b]['Ixt_mc'] for b in bvals]
            yt    = [d[b]['Iyt'] for b in bvals]
            if len(xtmc):
                plt.plot(xtmc, yt, ls={'reg':'-','sq':'-'}[sqmode], 
                         color=colordict[(mode,sqmode)],
                         marker='x', markersize=6, markerfacecolor='none', markeredgewidth=1,
                         label=mode + " (%s)" % sqmode)
                plt.fill_betweenx(yt, xtlb, xt, alpha=0.3, label=mode)
        plt.ylabel('$I(Y:T)$')
        plt.xlabel('$I(X:T)$')
        plt.legend(loc='lower right')
        plt.title(desc + ' - ' + {'tst':'Testing', 'trn':'Training'}[r])

        #plt.ylim([0, plt.ylim()[1]])
        #plt.xlim([0, plt.xlim()[1]])
        if False and desc == 'MNIST':
            maxv = 1.1*plt.ylim()[1]
            plt.plot([0, maxv], [0,maxv], 'k--', lw=1)
            plt.xlim([0, maxv])
            
        # Plot optimal Gaussian IB curve 
        #   doesn't seem to work ! -- suspect it has to do with dimensionality
        if False and desc=='Regression':
            with open('data/regression-10-100.pkl', 'rb') as f:
                data = pickle.load(f)
            mx = np.hstack([data[r+'_data'], data[r+'_labels']])
            xdims = data[r+'_data'].shape[1]
            xs, ys = entropy.get_gib_curve(np.cov(mx.T.astype('float64')), xdims)
            #print(xs, ys)
            plt.plot(xs/10, ys, c='k',label='Optimal')
    
    plt.ylim([0, 1.1*plt.ylim()[1]])
    plt.xlim([0, 1.1*plt.xlim()[1]])
    
    rng = [0.7*max(yt), 1.3*max(yt)]
    plt.xlim(rng)
    plt.ylim(rng)

plt.tight_layout()
# plt.savefig('plots/%s.pdf'%desc)
    
    